In [2]:
import pandas as pd
import numpy as np


In [19]:
# data
full_games_data
pred_sum
# data
# week_games_sum
points
week_games_sum
data[data['screen_name']=='Market'].groupby(['game_id'])['game_id'].count()
full_games_data

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,overtime,old_game_id,gsis,nfl_detail_id,pfr,pff,espn,away_rest,home_rest,away_moneyline,home_moneyline,spread_line,away_spread_odds,home_spread_odds,total_line,under_odds,over_odds,div_game,roof,surface,temp,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
0,1999_01_MIN_ATL,1999,REG,1,1999-09-12,Sunday,NaN,MIN,17.0,ATL,14.0,Home,-3.0,31.0,0.0,1999091210,598.0,NaN,199909120atl,NaN,190912001,7,7,NaN,NaN,-4.0,NaN,NaN,49.0,NaN,NaN,0,dome,astroturf,NaN,NaN,00-0003761,00-0002876,Randall Cunningham,Chris Chandler,Dennis Green,Dan Reeves,Gerry Austin,ATL00,Georgia Dome
1,1999_01_KC_CHI,1999,REG,1,1999-09-12,Sunday,NaN,KC,17.0,CHI,20.0,Home,3.0,37.0,0.0,1999091206,597.0,NaN,199909120chi,NaN,190912003,7,7,NaN,NaN,-3.0,NaN,NaN,38.0,NaN,NaN,0,outdoors,grass,80.0,12.0,00-0006300,00-0010560,Elvis Grbac,Shane Matthews,Gunther Cunningham,Dick Jauron,Phil Luckett,CHI98,Soldier Field
2,1999_01_PIT_CLE,1999,REG,1,1999-09-12,Sunday,NaN,PIT,43.0,CLE,0.0,Home,-43.0,43.0,0.0,1999091213,604.0,NaN,199909120cle,NaN,190912005,7,7,NaN,NaN,-6.0,NaN,NaN,37.0,NaN,NaN,1,outdoors,grass,78.0,12.0,00-0015700,00-0004230,Kordell Stewart,Ty Detmer,Bill Cowher,Chris Palmer,Bob McElwee,CLE00,Cleveland Browns Stadium
3,1999_01_OAK_GB,1999,REG,1,1999-09-12,Sunday,NaN,OAK,24.0,GB,28.0,Home,4.0,52.0,0.0,1999091208,602.0,NaN,199909120gnb,NaN,190912009,7,7,NaN,NaN,9.0,NaN,NaN,43.0,NaN,NaN,0,outdoors,grass,67.0,10.0,00-0005741,00-0005106,Rich Gannon,Brett Favre,Jon Gruden,Ray Rhodes,Tony Corrente,GNB00,Lambeau Field
4,1999_01_BUF_IND,1999,REG,1,1999-09-12,Sunday,NaN,BUF,14.0,IND,31.0,Home,17.0,45.0,0.0,1999091202,591.0,NaN,199909120clt,NaN,190912011,7,7,NaN,NaN,-3.0,NaN,NaN,45.5,NaN,NaN,1,dome,astroturf,NaN,NaN,00-0005363,00-0010346,Doug Flutie,Peyton Manning,Wade Phillips,Jim Mora,Ron Blum,IND99,RCA Dome
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6119,2021_18_CAR_TB,2021,REG,18,2022-01-09,Sunday,13:00,CAR,NaN,TB,NaN,Home,NaN,NaN,NaN,2022010911,58770.0,10160000-0587-704e-6295-0c29cd0a1891,202201090tam,NaN,401326596,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,outdoors,grass,NaN,NaN,NaN,NaN,NaN,NaN,Matt Rhule,Bruce Arians,NaN,TAM00,Raymond James Stadium
6120,2021_18_SEA_ARI,2021,REG,18,2022-01-09,Sunday,16:25,SEA,NaN,ARI,NaN,Home,NaN,NaN,NaN,2022010912,58771.0,10160000-0587-7113-a978-455fec3cda14,202201090crd,NaN,401326597,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,closed,grass,NaN,NaN,NaN,NaN,NaN,NaN,Pete Carroll,Kliff Kingsbury,NaN,PHO00,State Farm Stadium
6121,2021_18_KC_DEN,2021,REG,18,2022-01-09,Sunday,16:25,KC,NaN,DEN,NaN,Home,NaN,NaN,NaN,2022010913,58772.0,10160000-0587-7224-5e5e-c71409627e21,202201090den,NaN,401326598,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,outdoors,grass,NaN,NaN,NaN,NaN,NaN,NaN,Andy Reid,Vic Fangio,NaN,DEN00,Empower Field at Mile High
6122,2021_18_SF_LA,2021,REG,18,2022-01-09,Sunday,16:25,SF,NaN,LA,NaN,Home,NaN,NaN,NaN,2022010914,58773.0,10160000-0587-738e-2b05-95daf03cb563,202201090ram,NaN,401326599,7,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,dome,matrixturf,NaN,NaN,NaN,NaN,NaN,NaN,Kyle Shanahan,Sean McVay,NaN,LAX01,SoFi Stadium


In [1]:

pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 400)

#Prediction data CSV
data = pd.read_csv('https://raw.githubusercontent.com/nflverse/nfldata/master/data/predictions.csv')

#Game results CSV
full_games_data = pd.read_csv('https://raw.githubusercontent.com/nflverse/nfldata/master/data/games.csv')

#Get median prediction for each game (not including Market)
#Some people also choose to exclude picks that are at 50% (not shown here)
pred_sum = data.loc[(data.screen_name!='Market')].groupby('game_id')[['prediction']].median()

#Market prediction for each game
pred_sum['market'] = data.loc[data.screen_name=='Market'].groupby('game_id')[['prediction']].sum()

#Full games data with results for this season
week_games_sum = full_games_data.loc[full_games_data.season==2021]

#Only keep select columns
week_games_sum = week_games_sum[['game_id','away_team','home_team','result']]

#If result column is > 0 then home team won
week_games_sum['winner'] = week_games_sum['result']
week_games_sum.loc[week_games_sum.result > 0, 'winner'] = week_games_sum.home_team
week_games_sum.loc[week_games_sum.result < 0, 'winner'] = week_games_sum.away_team

#Join to pred_sum
week_games_sum = week_games_sum.merge(pred_sum, left_on='game_id', right_index=True)

#Column for what the median user predicted winner was
week_games_sum['pred_winner'] = np.nan
week_games_sum.loc[week_games_sum.prediction > 50, 'pred_winner'] = week_games_sum.home_team
week_games_sum.loc[week_games_sum.prediction <= 50, 'pred_winner'] = week_games_sum.away_team

#Dataframe with every prediction as a row and the game result
points = data.merge(full_games_data[['game_id','home_team','away_team','week','season','result']],
                    left_on='game_id', right_on='game_id')

#Calculate points result for each prediction
points['pts'] = np.nan

#Formula for points
#M * (25 - (100 * (P/100 - R)^2))

#Neutral pick 50% gets 0 points
points.loc[points.prediction == 50, 'pts'] = 0

#Predicts Home Win, Actual Home Win
points.loc[(points.prediction > 50) & (points.result > 0), 'pts'] = 1 * (25 - (100 * (points.prediction/100 - 1)**2))

#Predicts Home Win, Actual Home Loss
points.loc[(points.prediction > 50) & (points.result < 0), 'pts'] = 1 * (25 - (100 * (points.prediction/100 - 0)**2))

#Create new column for away prediction
points['away_pred'] = np.nan

#Flip away predictions to the other side of 50 (i.e. 30 becomes 70)
points.loc[points.prediction < 50, 'away_pred'] = 100 - points.prediction 

#Predicts Away Win, Actual Away Win
points.loc[(points.away_pred > 50) & (points.result < 0), 'pts'] = 1 * (25 - (100 * (points.away_pred/100 - 1)**2))

#Predicts Away Win, Actual Away Loss
points.loc[(points.away_pred > 50) & (points.result > 0), 'pts'] = 1 * (25 - (100 * (points.away_pred/100 - 0)**2))

#Get each user's points
points.groupby('screen_name')[['pts']].sum().sort_values('pts')

#Get each game's aggregate points won/lost
points.groupby(['game_id','result','home_team','away_team'], as_index=False)[['pts']].sum()

,game_id,result,home_team,away_team,pts
0,2021_01_ARI_TEN,-25.0,TEN,ARI,-5297.83
1,2021_01_BAL_LV,6.0,LV,BAL,-14036.12
2,2021_01_CHI_LA,20.0,LA,CHI,8499.02
3,2021_01_CLE_KC,4.0,KC,CLE,6990.04
4,2021_01_DAL_TB,2.0,TB,DAL,8516.11
5,2021_01_DEN_NYG,-14.0,NYG,DEN,2343.13
6,2021_01_GB_NO,35.0,NO,GB,-11907.86
7,2021_01_JAX_HOU,16.0,HOU,JAX,-7703.77
8,2021_01_LAC_WAS,-4.0,WAS,LAC,201.70
9,2021_01_MIA_NE,-1.0,NE,MIA,-3428.13


#### 2021-11-07 use elo as predictions
- calc elo for each game
- output prediction probability
- ally games results, compare to elo predictions
- see how good prediction was?
    - what measure? error rate?
    - how to combine with probability prediction e.g 65% chance of Browns win
